<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets evaluate accelerate wandb
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


# Token classification

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/wVHdVlPScxA?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Token classification assigns a label to individual tokens in a sentence. One of the most common token classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This guide will show you how to:

1. Finetune [DistilBERT](https://huggingface.co/distilbert/distilbert-base-uncased) on the [WNUT 17](https://huggingface.co/datasets/wnut_17) dataset to detect new entities.
2. Use your finetuned model for inference.

<Tip>

To see all architectures and checkpoints compatible with this task, we recommend checking the [task-page](https://huggingface.co/tasks/token-classification).

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate seqeval
```

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load WNUT 17 dataset

Start by loading the WNUT 17 dataset from the 🤗 Datasets library:

In [3]:
from datasets import load_dataset

#wnut = load_dataset("wnut_17", data_dir="wnut_17", revision="refs/convert/parquet")
gn = load_dataset("unimelb-nlp/wikiann", "gn")
es = load_dataset("unimelb-nlp/wikiann", "es")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

gn/validation-00000-of-00001.parquet:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

gn/test-00000-of-00001.parquet:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

gn/train-00000-of-00001.parquet:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

...

es/test-00000-of-00001.parquet:   0%|          | 0.00/608k [00:00<?, ?B/s]

es/train-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [4]:
from random import shuffle
from datasets import concatenate_datasets

# Concatenate train, validation, and test sets from both languages
combined_dataset = concatenate_datasets(
    [gn["train"], gn["validation"], gn["test"],
     es["train"], es["validation"], es["test"]]
)

# Split the combined dataset into training and the rest (which will be split again into validation and test)
train_testvalid = combined_dataset.train_test_split(
    test_size=0.3, shuffle=True, seed=42
)

# Split the 'test' part of the previous split into validation and test sets
wnut = train_testvalid["test"].train_test_split(
    test_size=0.75, shuffle=True, seed=42 # Split 75/25 to get validation and test
)

# Rename the splits to 'train', 'validation', and 'test'
wnut["validation"] = wnut.pop("train") # Rename the first part of the second split to validation
wnut["train"] = train_testvalid["train"]
wnut["test"] = wnut.pop("test") # Rename the second part of the second split to test

wnut

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 3022
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 28210
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 9068
    })
})

Then take a look at an example:

In [5]:
wnut["train"][0]

{'tokens': ['San',
  'Francisco',
  ',',
  'California',
  ',',
  'Estados',
  'Unidos',
  '.'],
 'ner_tags': [5, 6, 0, 5, 0, 5, 6, 0],
 'langs': ['es', 'es', 'es', 'es', 'es', 'es', 'es', 'es'],
 'spans': ['LOC: San Francisco', 'LOC: California', 'LOC: Estados Unidos']}

Each number in `ner_tags` represents an entity. Convert the numbers to their label names to find out what the entities are:

In [6]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

The letter that prefixes each `ner_tag` indicates the token position of the entity:

- `B-` indicates the beginning of an entity.
- `I-` indicates a token is contained inside the same entity (for example, the `State` token is a part of an entity like
  `Empire State Building`).
- `0` indicates the token doesn't correspond to any entity.

In [8]:
ner = zip(wnut["train"][0]["tokens"], wnut["train"][0]["ner_tags"])
for item in ner:
  print(f"{item} => {label_list[item[1]]}")

('San', 5) => B-LOC
('Francisco', 6) => I-LOC
(',', 0) => O
('California', 5) => B-LOC
(',', 0) => O
('Estados', 5) => B-LOC
('Unidos', 6) => I-LOC
('.', 0) => O


## Preprocess

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/iY2AZYdZAr0?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The next step is to load a mBERT tokenizer to preprocess the `tokens` field:

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-v-base")#"mmaguero/multilingual-bert-gn-base-cased") #"distilbert/distilbert-base-uncased")

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/61.4M [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [10]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>',
 '▁San',
 '▁Francisco',
 '▁',
 ',',
 '▁California',
 '▁',
 ',',
 '▁Estados',
 '▁Unidos',
 '▁',
 '.',
 '</s>']

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function (see [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)).
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [13]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3022 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/28210 [00:00<?, ? examples/s]

Map:   0%|          | 0/9068 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [15]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=afe9af32f6fff970417f1ef941577212d52cf71232a04d971178093b1805f21d
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [16]:
import evaluate

seqeval = evaluate.load("seqeval")

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [17]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [18]:
id2label = {
    0:'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5:'B-LOC', 6:'I-LOC'
}
label2id = {
    'O':0, 'B-PER':1, 'I-PER':2, 'B-ORG':3, 'I-ORG':4, 'B-LOC':5, 'I-LOC':6
}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [19]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "facebook/xlm-v-base", # "mmaguero/multilingual-bert-gn-base-cased", # "distilbert/distilbert-base-uncased",
    num_labels=7, id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at facebook/xlm-v-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [20]:
training_args = TrainingArguments(
    output_dir="wikiann-xlm-v-base",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # 3 a 10 o más epochs
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit = 3,
    push_to_hub=True, # True para subir a HF
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mmaguero to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.255000,0.195131,0.888829,0.901662,0.895199,0.946546
2,0.188200,0.165567,0.901374,0.911196,0.906258,0.954410
3,0.145400,0.159881,0.906989,0.919096,0.913002,0.957545


TrainOutput(global_step=5292, training_loss=0.24728386274180625, metrics={'train_runtime': 3222.4696, 'train_samples_per_second': 26.262, 'train_steps_per_second': 1.642, 'total_flos': 1016656711825788.0, 'train_loss': 0.24728386274180625, 'epoch': 3.0})

In [21]:
trainer.evaluate(tokenized_wnut["test"])

{'eval_loss': 0.18721306324005127,
 'eval_precision': 0.9064130915524105,
 'eval_recall': 0.9119793520825916,
 'eval_f1': 0.9091877024089438,
 'eval_accuracy': 0.9524881860211202,
 'eval_runtime': 14.2399,
 'eval_samples_per_second': 636.804,
 'eval_steps_per_second': 39.818,
 'epoch': 3.0}

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [22]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-v-base/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...e/sentencepiece.bpe.model:  92%|#########1| 16.7MB / 18.2MB            

  ...xlm-v-base/tokenizer.json:  14%|#3        | 8.35MB / 61.3MB            

  ...-v-base/model.safetensors:   0%|          | 8.37MB / 3.11GB            

  ...470786.af7912a45715.850.0: 100%|##########| 9.19kB / 9.19kB            

  ...474047.af7912a45715.850.1: 100%|##########|   560B /   560B            

CommitInfo(commit_url='https://huggingface.co/mmaguero/wikiann-xlm-v-base/commit/c8f6c4dcf0489a5ec3bba8cdc3eed1e4a48532bb', commit_message='End of training', commit_description='', oid='c8f6c4dcf0489a5ec3bba8cdc3eed1e4a48532bb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mmaguero/wikiann-xlm-v-base', endpoint='https://huggingface.co', repo_type='model', repo_id='mmaguero/wikiann-xlm-v-base'), pr_revision=None, pr_num=None)

<Tip>

For a more in-depth example of how to finetune a model for token classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [23]:
text = "Golden State Warriors ha'e peteĩ equipo profesional de baloncesto USA pegua oĩva San Francisco-pe."
text_tokens = text.split()

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [24]:
from transformers import pipeline

classifier = pipeline("ner", model="mmaguero/wikiann-xlm-v-base", aggregation_strategy="max")

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.11G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/61.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


The `aggregation_strategy` parameter in the `pipeline("ner", ...)` function controls how subword predictions are combined into full word predictions.

*   When `aggregation_strategy="max"` (the default for NER), the pipeline assigns the entity label with the *maximum* score among all the subwords belonging to a word to that entire word. This is a common and often effective strategy for NER tasks.

Here are some other common `aggregation_strategy` options you might encounter or use in different token classification pipelines:

*   `"none"`: No aggregation is performed. Each subword receives its own prediction. This is useful if you need fine-grained control or want to analyze subword-level predictions directly.

*   `"simple"`: This strategy typically aggregates subwords that form a complete word. For NER, it usually takes the first subword's prediction for a word, but the exact behavior can vary slightly depending on the specific pipeline implementation. It aims to simplify the output to one prediction per full word.

*   `"average"`: This strategy would average the scores of all subwords belonging to a word to determine the final entity for that word. This can sometimes lead to smoother predictions by incorporating information from all subword pieces.

For Named Entity Recognition, `"max"` is often a good default choice because it helps in identifying entities even if some subwords have lower confidence scores within the same entity.

In [25]:
classifier(text_tokens)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:490: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


[[{'entity_group': 'LOC',
   'score': np.float32(0.8637679),
   'word': 'Golden',
   'start': 0,
   'end': 6}],
 [{'entity_group': 'LOC',
   'score': np.float32(0.95016104),
   'word': 'State',
   'start': 0,
   'end': 5}],
 [{'entity_group': 'ORG',
   'score': np.float32(0.8934903),
   'word': 'Warriors',
   'start': 0,
   'end': 8}],
 [{'entity_group': 'ORG',
   'score': np.float32(0.724045),
   'word': "ha'e",
   'start': 0,
   'end': 4}],
 [],
 [{'entity_group': 'ORG',
   'score': np.float32(0.83224255),
   'word': 'equipo',
   'start': 0,
   'end': 6}],
 [{'entity_group': 'PER',
   'score': np.float32(0.5052041),
   'word': 'profesional',
   'start': 0,
   'end': 11}],
 [],
 [{'entity_group': 'ORG',
   'score': np.float32(0.87905157),
   'word': 'baloncesto',
   'start': 0,
   'end': 10}],
 [{'entity_group': 'LOC',
   'score': np.float32(0.98684955),
   'word': 'USA',
   'start': 0,
   'end': 3}],
 [],
 [],
 [{'entity_group': 'LOC',
   'score': np.float32(0.6776837),
   'word': 'S

In [26]:
wnut["test"][0]["tokens"], wnut["test"][0]["ner_tags"]

(["'", "''", 'Rise', 'Against', "''", "'"], [0, 0, 3, 4, 0, 0])

In [27]:
classifier(wnut["test"][0]["tokens"])

[[],
 [],
 [{'entity_group': 'PER',
   'score': np.float32(0.6761217),
   'word': 'Rise',
   'start': 0,
   'end': 4}],
 [{'entity_group': 'ORG',
   'score': np.float32(0.82620037),
   'word': 'Against',
   'start': 0,
   'end': 7}],
 [],
 []]

In [28]:
text = "Mañana voy a ir a Asunción y luego iré a Luque a la casa de mi tío Carlos."
text_tokens = text.split()
text_tokens, classifier(text_tokens)

(['Mañana',
  'voy',
  'a',
  'ir',
  'a',
  'Asunción',
  'y',
  'luego',
  'iré',
  'a',
  'Luque',
  'a',
  'la',
  'casa',
  'de',
  'mi',
  'tío',
  'Carlos.'],
 [[{'entity_group': 'ORG',
    'score': np.float32(0.54235977),
    'word': 'Mañana',
    'start': 0,
    'end': 6}],
  [{'entity_group': 'ORG',
    'score': np.float32(0.45536405),
    'word': 'voy',
    'start': 0,
    'end': 3}],
  [],
  [],
  [],
  [{'entity_group': 'LOC',
    'score': np.float32(0.9836353),
    'word': 'Asunción',
    'start': 0,
    'end': 8}],
  [],
  [],
  [],
  [],
  [{'entity_group': 'LOC',
    'score': np.float32(0.9610221),
    'word': 'Luque',
    'start': 0,
    'end': 5}],
  [],
  [],
  [{'entity_group': 'LOC',
    'score': np.float32(0.5071324),
    'word': 'casa',
    'start': 0,
    'end': 4}],
  [],
  [],
  [{'entity_group': 'PER',
    'score': np.float32(0.6376002),
    'word': 'tío',
    'start': 0,
    'end': 3}],
  [{'entity_group': 'PER',
    'score': np.float32(0.96253246),
    'wo

In [29]:
text = "Ko'ero aháta Paraguay-pe ha upé ahásata Luque-pe che tío Kálo ogápe."
text_tokens = text.split()
text_tokens, classifier(text_tokens)

(["Ko'ero",
  'aháta',
  'Paraguay-pe',
  'ha',
  'upé',
  'ahásata',
  'Luque-pe',
  'che',
  'tío',
  'Kálo',
  'ogápe.'],
 [[{'entity_group': 'LOC',
    'score': np.float32(0.64427567),
    'word': "Ko'ero",
    'start': 0,
    'end': 6}],
  [{'entity_group': 'LOC',
    'score': np.float32(0.8093514),
    'word': 'aháta',
    'start': 0,
    'end': 5}],
  [],
  [],
  [],
  [{'entity_group': 'LOC',
    'score': np.float32(0.9188303),
    'word': 'ahásata',
    'start': 0,
    'end': 7}],
  [{'entity_group': 'LOC',
    'score': np.float32(0.9761203),
    'word': 'Luque-pe',
    'start': 0,
    'end': 8}],
  [],
  [{'entity_group': 'PER',
    'score': np.float32(0.6376002),
    'word': 'tío',
    'start': 0,
    'end': 3}],
  [{'entity_group': 'LOC',
    'score': np.float32(0.8301375),
    'word': 'Kálo',
    'start': 0,
    'end': 4}],
  []])

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mmaguero/wikiann-multilingual-bert-gn-base-cased")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [ ]:
import torch
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("mmaguero/wikiann-multilingual-bert-gn-base-cased")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [ ]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
inputs, predicted_token_class

({'input_ids': tensor([[  101, 13744, 13995, 10219, 12556, 10157,   169, 10478,   169, 53799,
            193, 17391, 10478, 10333,   169, 23859, 11189,   169, 10109, 12088,
          10104, 12132, 94354, 12050,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]])},
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-ORG',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O'])